In [ ]:
!pip install -q faiss-cpu sentence-transformers PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.5 MB/s eta 0:00:00


#Uploading the relevant Books to add in knowledge base

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 090310.pdf to 090310.pdf
Saving SLS.pdf to SLS.pdf
Saving Ian Goodfellow, Yoshua Bengio, Aaron Courville - Deep Learning (2017, MIT).pdf to Ian Goodfellow, Yoshua Bengio, Aaron Courville - Deep Learning (2017, MIT).pdf
Saving ML Machine Learning-A Probabilistic Perspective.pdf to ML Machine Learning-A Probabilistic Perspective.pdf
Saving machine-learning-yearning.pdf to machine-learning-yearning.pdf
Saving Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf to Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf
Saving ESLII_print12_toc.pdf to ESLII_print12_toc.pdf
Saving book1.pdf to book1.pdf
Saving book2.pdf to book2.pdf


#Importing Relevant Libraries

In [ ]:
import faiss
import numpy as np
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer

# Extracting Text from PDFs

In [ ]:
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""
    return text

# Breaking Down the Text into Chunks

In [ ]:
def create_chunks(text, chunk_size=500):
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0

    for word in words:
        current_chunk.append(word)
        current_length += 1
        if current_length >= chunk_size:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_length = 0

    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

#Vector Encoding

In [ ]:
def create_embeddings(chunks, model):
    return model.encode(chunks)

#Indexing for Fast Access

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")
index = faiss.IndexFlatL2(384)  # 384 is embedding dimension
all_chunks = []


#Processing of books one by one

In [ ]:
for filename in uploaded.keys():
    print(f"Processing {filename}")
    text = extract_text_from_pdf(filename)
    chunks = create_chunks(text)
    embeddings = create_embeddings(chunks, model)
    index.add(np.array(embeddings).astype("float32"))
    all_chunks.extend(chunks)

Processing 090310.pdf
Processing SLS.pdf
Processing Ian Goodfellow, Yoshua Bengio, Aaron Courville - Deep Learning (2017, MIT).pdf
Processing ML Machine Learning-A Probabilistic Perspective.pdf
Processing machine-learning-yearning.pdf
Processing Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf
Processing ESLII_print12_toc.pdf
Processing book1.pdf
Processing book2.pdf


#creating the Index File

In [ ]:
faiss.write_index(index, "faiss_index.index")

#Storing the Chunks

In [ ]:
with open("chunks.txt", "w", encoding="utf-8") as f:
    for chunk in all_chunks:
        f.write(f"{chunk}\n\n")

#To download the knowledge base

In [ ]:
files.download("faiss_index.index")
files.download("chunks.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Chat BOT Implementation

## Import packages to install

In [1]:
!pip install -q sentence-transformers faiss-cpu transformers torch PyPDF2 sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 17.5 MB/s eta 0:00:00


## Relevant libraries to import

In [2]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import pipeline
import torch

#Upload Chunks and Upload faiss_index


In [4]:
from google.colab import files

uploaded = files.upload()  # Upload both files here

Saving faiss_index.index to faiss_index.index
Saving chunks.txt to chunks.txt


#Model to transform queries into relevant form to search

In [5]:
model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#initialize index

In [6]:
index = faiss.read_index("faiss_index.index")

#initiailze Chunks i.e. open chunks

In [7]:
with open("chunks.txt", "r", encoding="utf-8") as f:
    chunks = f.read().split("\n\n")

#To output answer properly

In [8]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


#Function to process input and get output from knowledge base

In [9]:
def process_input(user_input):
    try:
        if not user_input.strip():
            return "Error: Input cannot be empty."

        # Create embedding
        query_embedding = model.encode([user_input])
        query_embedding = np.array(query_embedding, dtype=np.float32)

        # Search FAISS index
        D, I = index.search(query_embedding, k=1)

        if I[0][0] == -1 or index.ntotal == 0:
            return "Sorry, I couldn't find relevant information."

        # Get the most relevant chunk
        best_chunk = chunks[I[0][0]]

        # Summarize the result
        summary = summarizer(best_chunk, max_length=100, min_length=30, do_sample=False)
        return summary[0]['summary_text']

    except Exception as e:
        return f"An error occurred: {str(e)}"

# Loop for Chatbot

In [ ]:
def chat():
    print("🤖 Chatbot ready! Type 'exit' to quit.\n")
    while True:
        query = input("You: ")
        if query.lower() in ["exit", "quit"]:
            print("Chatbot: Bye ! 👋")
            break
        response = process_input(query)
        print("Chatbot:", response)

chat()


🤖 Chatbot ready! Type 'exit' to quit.

You: linear regression
Chatbot:  Linear regression is often used in business, government, and other scenarios. Some common practical applications of linear regression in the real world include the following. After presenting possible linear regression use cases, the foundations of the technique are provided.
You: knn
Chatbot: The K-nearest-neighbour algorithm can be used to analyse oil data. It is based on the K=1 rule, where a test point is assigned to the same class as the nearest point from the training set. In Figure 2.28, we show the results of applying the algorithm to the oil data set for various values of K.
